In [80]:
import sys
import pandas as pd
sys.path.insert(1, 'tensorflow')
from mytensorflow import *
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfdt

In [71]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    "../data/catsdogs",
    validation_split=0.2, 
    subset="both", 
    seed=42,
    image_size=(200,200), 
    batch_size=32, 
)

Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Using 400 files for validation.


In [72]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
    ]
)

In [73]:
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label)
)

In [ ]:
net = Network()
net.add(ConvolutionalLayer((200,200,3), 3, 5))
net.add(ActivationLayer(Sigmoid.callFunction, Sigmoid.callDerivative))
net.add(FlattenLayer(None, None))
net.add(FCLayer(200, 200))
net.add(ActivationLayer(TangH.callFunction, TangH.callDerivative))
net.add(FCLayer(3, 8))
net.add(ActivationLayer(Relu.callFunction, Relu.callDerivative))
net.add(FCLayer(8, 1))
net.add(ActivationLayer(Sigmoid.callFunction, Sigmoid.callDerivative))


net.compile(BinaryCrossEntropy.fun, BinaryCrossEntropy.derivative)
net.fit(train_ds, test_ds, epochs=1000, learning_rate=0.1)

TypeError: '_MapDataset' object is not subscriptable